In [3]:
import numpy as np

In [6]:
phi = np.load("Phi_201snap.npy")
phi

array([[[ 1.72257e-09, -2.98010e-09,  3.34862e-09, ..., -1.28920e-01,
         -1.56465e-01, -2.22917e-01],
        [ 1.72257e-09, -2.98010e-09,  3.34862e-09, ..., -1.28920e-01,
         -1.56465e-01, -2.22917e-01],
        [ 1.72257e-09, -2.98010e-09,  3.34862e-09, ..., -1.28920e-01,
         -1.56465e-01, -2.22917e-01],
        ...,
        [ 1.72257e-09, -2.98010e-09,  3.34862e-09, ..., -1.28920e-01,
         -1.56465e-01, -2.22917e-01],
        [ 1.72257e-09, -2.98010e-09,  3.34862e-09, ..., -1.28920e-01,
         -1.56465e-01, -2.22917e-01],
        [ 1.72257e-09, -2.98010e-09,  3.34862e-09, ..., -1.28920e-01,
         -1.56465e-01, -2.22917e-01]],

       [[ 1.58749e-09, -3.01799e-09,  3.29686e-09, ..., -1.26816e-01,
         -1.69784e-01, -2.30471e-01],
        [ 1.58924e-09, -3.01753e-09,  3.29744e-09, ..., -1.26791e-01,
         -1.69586e-01, -2.30515e-01],
        [ 1.59099e-09, -3.01708e-09,  3.29802e-09, ..., -1.26766e-01,
         -1.69388e-01, -2.30558e-01],
        ...,
